In [20]:
using FastGaussQuadrature, LinearAlgebra

In [120]:
nodes, weights = gausslegendre( 100 );
weights.*=0.5
nodes .+=1
nodes .*=0.5;

nodesmany, weightsmany = gausslegendre( 1000 );
weightsmany.*=0.5
nodesmany .+=1
nodesmany .*=0.5;

In [121]:
function xi(eta,u)
    q = clamp((1.0+eta)/(1.0-eta),-1e80,1e80)
    return (log(q)+u*1im)/(eta+tan(u/2)*1im)    
end

xi (generic function with 1 method)

In [122]:
function phi_pt1(r,t)
    eta = r/t
    q = clamp((1.0+eta)/(1.0-eta),-1e80,1e80)
    return exp(-t)/(4*pi*r*t^2)*t*log(q)
end

phi_pt1 (generic function with 1 method)

In [123]:
function integrand_pt(u,eta,t)
    return sec(u/2)^2*real((eta+1im*tan(u/2))*xi(eta,u)^3*exp(t/2*(1-eta^2)*xi(eta,u)))
end

integrand_pt (generic function with 1 method)

In [124]:
function phi_pt(r,t)
    r = clamp(r,1e-10,1e80)
    eta = r/t
    g = 0.0
    if eta<1.0
        g = dot(weightsmany, integrand_pt.(nodesmany*pi,eta,t))
    end
    f = 1/(2*pi)*exp(-t) /(4*pi*r*t^2)*(t/2)^2*(1-eta^2)*g
    f = f + phi_pt1(r,t)
    return f
end

phi_pt (generic function with 1 method)

In [125]:
function phi_l_single(eta,t)
    argument = t*sqrt.(eta^2 .+(nodes.*sqrt(1 .-eta.^2)).^2)
    f = dot(weights, phi_pt.(argument,t))
    phi_l0 = exp(-t)/(2*pi*t^2) / sqrt(1-eta^2)
    f = phi_l0 + (2*t)*f
    return f
end

phi_l_single (generic function with 1 method)

In [126]:
function phi_l(t,r)
    eta = r/t
    if eta<1.0
        return phi_l_single(eta,t)
    else
        return 0.0
    end
end

phi_l (generic function with 1 method)

In [127]:
function integrand(tau,r,sigmas,sigmaa)
   return  sigmas*exp(-(sigmaa+sigmas)*tau)*phi_l(tau,r-tau)
end

integrand (generic function with 1 method)

In [128]:
r = 0.5
t = 1.0
sigmaa = 1.0
sigmas = 1.0


f(tau) = integrand(tau,r,sigmas,sigmaa)
print(dot(weightsmany,f.(nodesmany*t)))

0.385941379533749